In [3]:
import pandas as pd
import re
import numpy as np

## On va ajouter de l'Open Data pour avoir les latitudes, longitudes et les régions complètes.

In [8]:
jobs = pd.read_csv('Jobs.csv')
cities = pd.read_csv('cities.csv',sep=';')

FileNotFoundError: [Errno 2] No such file or directory: 'Jobs.csv'

In [ ]:
cities['ville'] = cities['ville'].str.upper()
jobs['ville'] = jobs['ville'].str.upper()

In [ ]:
#on enlève les doublons
cities = cities.drop_duplicates(subset='ville')
jobs = jobs.drop_duplicates()

In [ ]:
columns_to_drop = ['commune', 'latitude', 'longitude', 'codePostal']
jobs = jobs.drop(columns=columns_to_drop)
df = pd.merge(jobs, cities, how='left', on='ville')

## Transformation variable "Salaire"

In [ ]:
#transformation des donnes type "Annuel de 40,00 Euros à 60,00 Euros" en "Annuel de 40000 Euros à 60000 Euros":
#transformation des données de type 'Mensuel de 1136,00 Euros sur 12 mois':
df['salaire'] = df['salaire'].replace('Mensuel de 1136,00 Euros sur 12 mois', 'Annuel de 13632,00')
df['salaire'] = df['salaire'].replace('Mensuel de 4000,00 Euros à 5000,00 Euros sur 12 mois', 'Annuel de 48000,00 Euros à 60000,00 Euros sur 12 mois')
df['salaire'] = df['salaire'].replace('Annuel de 40,00 Euros à 60,00 Euros', 'Annuel de 40000,00 Euros à 60000,00 Euros')
df['salaire'] = df['salaire'].replace('Annuel de 50,00 Euros à 70,00 Euros', 'Annuel de 50000,00 Euros à 70000,00 Euros')
df['salaire'] = df['salaire'].replace('Annuel de 26000 Euros à 36000 Euros sur 12 mois', 'Annuel de 26000,00 Euros à 36000,00 Euros sur 12 mois')
df['salaire'] = df['salaire'].astype(str)

In [ ]:
def extract_salaire_min(salaire):
    match = re.search(r'(\d+,\d+)', salaire)
    return float(match.group(1).replace(',', '.')) if match else None

def extract_salaire_max(salaire):
    match = re.search(r'(\d+,\d+)(?: Euros à (\d+,\d+))?', salaire)
    return float(match.group(2).replace(',', '.')) if match and match.group(2) else None

df['salaire_min_annuel'] = df['salaire'].apply(extract_salaire_min)
df['salaire_max_annuel'] = df['salaire'].apply(extract_salaire_max)

In [ ]:
df['salaire_min_annuel'] = pd.to_numeric(df['salaire_min_annuel'].replace('nan', np.nan), errors='coerce')
df['salaire_max_annuel'] = pd.to_numeric(df['salaire_max_annuel'].replace('nan', np.nan), errors='coerce')
df['salaire'] = df['salaire'].replace('nan', np.nan)

In [ ]:
#Création de la variable 'salaire_annuel_mean':
df['salaire_annuel_mean'] = np.where(df['salaire_max_annuel'].notnull(),
                                     (df['salaire_min_annuel'] + df['salaire_max_annuel']) / 2,
                                     df['salaire_min_annuel'])

## Renommage des colonnes

In [ ]:
# renommer les colonnes (pour la base de donnée)
df = df.rename(columns={'dateCreation':'date_creation', 'qualificationLibelle':'qualification_libelle', 'experienceOuiNon':"experience", 'typeContrat':'type_contrat', 'secteurActiviteLibelle':'secteur_activite', 'codePostal':'code_postal', 'department_name':'departement_nom', 'profils':'profil' })

## Creation de la colonne 'poste'

In [ ]:
def categorize_poste(row):
    intitule_lower = row['intitule'].lower()
    if 'data scientist' in intitule_lower or 'ingénieur data scientist' in intitule_lower:
        return 'data scientist'
    elif 'data engineer' in intitule_lower or 'engineer' in intitule_lower or 'ingénieur' in intitule_lower or 'engineering' in intitule_lower :
        return 'data engineer'


df['poste'] = df.apply(categorize_poste, axis=1)

In [ ]:
# suppression des postes qui ne sont ni data scientist ni data ingénieur
df.dropna(subset=['poste'], inplace=True)

## Enrichissement de la variable 'profil'

In [ ]:
# Fonction pour extraire le contenu après 'profil recherché'
def extraire_profil_recherche(description):
    # Utilisation d'une expression régulière pour trouver le texte après 'profil recherché'
    match = re.search(r'(?i)profil recherché', description)
    
    # Si la correspondance est trouvée, retourner le contenu après la correspondance
    if match:
        return description[match.end():].strip()
 

# Appliquer la fonction à la colonne 'description_offre', dont les lignes ne contiennent pas de données dans la colonne 'profil'
# Créer la nouvelle colonne 'profil_recherche'
mask = df['profil'].isnull() | (df['profil'] == '')
df.loc[mask, 'profil_recherche'] = df.loc[mask, 'description_offre'].apply(extraire_profil_recherche)


In [ ]:
# Fusionner le contenu de 'profil' et 'profil_recherche' dans une nouvelle variable 'profil2'
df['profil2'] = df.apply(lambda row: row['profil'] + ' ' + row['profil_recherche'] if pd.notnull(row['profil']) and pd.notnull(row['profil_recherche']) else row['profil'] if pd.notnull(row['profil']) else row['profil_recherche'], axis=1)

In [ ]:
# Supprimer les colonnes 'profil_recherche' et 'profil' et renommer 'profil2' en 'profil':
df = df.drop(columns=['profil_recherche', 'profil'])
df.rename(columns={'profil2': 'profil'}, inplace=True)

In [ ]:
df.to_csv('Jobs_V4.csv', index=False)